In [1]:
import time
import pickle
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm

// codice che usavo prima (importavo tutto il codice pesante, ora importo solo il corpus finale!!)

time1 = time.time()
import TextImport
time2 = time.time()
print((time2-time1))

Primo Import iniziato alle 13:21 , finito alle 13:33 (800 secondi)

Secondo import 580 secondi

Terzo Import 581 secondi

La cosa strana è che runnando TextImport su visualStudio ci mette 250 secondi! (ovvero runnare è più
veloce che importare!?)

# Rappresentazione del corpus usando i word embeddings:


In [2]:
with open('corpus_filtrato15_11.pkl', 'rb') as file:
    # Carica i dati dal file
    corpus_filtrato = pickle.load(file) # corpus filtrato è il corpus senza le parole inutili

Corpus_filtrato è stato prodotto dal file "TextImportOriginale" nella cartella "word embeddings", vado lì se voglio modificare qualcosa!

In [3]:
with open("my_vecs_preProcessingSenzaParoleInutili.p", "rb") as file:
    word2Vec_model = pickle.load(file)

Vedo Word2VecDatiIMPORT nella cartella WordEmbedding per avere un'idea di come usare gli embedding e di come sono stati fatti.

In seguito spiego la corrispondenza univoca fra i vettori numerici (embeddings) delle parole del modello e le parole presenti in Vocabolario preso da "textImport"

In [4]:
word_names = word2Vec_model.wv.index_to_key # word names è la stringa di parola corrispondente all'embedding numerico
# (Traduzione Embedding -> stringa)
word_vectors = [word2Vec_model.wv[word] for word in word_names]
# word_vectors è l'embedding relativo alle parole di word_names



Osservazione:

In [7]:
# ParoleMancanti = [parola for parola in word_names if parola not in TextImport.Vocabolario]
#print(len(word_names))
#print(len(TextImport.ListaParoleInutili))
#print(len(word_names)-len(TextImport.ListaParoleInutili))
#len(TextImport.Vocabolario)

# Risultato importantissimo, prima mi veniva che word_names (ovvero il vettore delle parole sputate dal modello di gensim) e 
# il mio Vocabolario avessero lunghezza lievemente diversa. Questo era perchè in precedenza, sbagliando avevo allenato il modello 
# senza togliere le parole inutili. Ora invece tutto torna!

Ora traduco il corpus (lista di liste di parole ovvero lista di documenti) traducendo ogni parola in embedding!

In [5]:
corpus_filtrato_embedding = []
for doc in corpus_filtrato:
    doc_embedding = []
    for parola in doc:
        doc_embedding.append(word2Vec_model.wv[parola])
    corpus_filtrato_embedding.append(doc_embedding)
    

In [9]:
#corpus_filtrato_embedding[0][0]
# ho fatto diverse prove e sembra proprio che tutto il procedimento di traduzione sia andato a buon fine!

Corpus_filtrato_embedding è il corpus formato solo dalle parole utili, dove ogni parola è un vettore di dim 10 (embedding)

# Costruzione grafo:


Iniziamo con il modello più semplice possibile, ovvero i collegamenti di una parola sono solo con la precedente e con la successiva


Per capire come costruire il grafo prendo modelli->INFOGRAPH->data-> MUTAG e ne copio la struttura.

Consideriamo il primo documento, da trasformare in un grafo, da questo tutti gli altri documenti verranno modificati con lo stesso procedimento, l'unica differenza è la lunghezza del documento ma non è un problema.

In [6]:
Doc0 = corpus_filtrato_embedding[0]
Doc0 = np.array(Doc0) # meglio averceli in numpy per fare le operazionui!

In [7]:
# Apre un file in modalità scrittura
with open('Graph0_A.txt', 'w') as file:
    # Lista di coppie di valori
    values = []
    for i in range(len(Doc0)):
        if i == 0:
            val = (i,i+1)
            values.append(val)
        elif i == len(Doc0)-1:
            val = (i,i-1)
            values.append(val)
        else:
            val1= (i,i-1)
            values.append(val1)
            val2 = (i,i+1)
            values.append(val2)

    # Itera sulla lista e scrive le coppie nel file
    for value in values:
        file.write(f"{value[0]}, {value[1]}\n")
        
## abbiamo quindi creato la lista di adiacenza "A"

In [21]:
with open('E3C_node_attributes.txt', 'w') as file:
    values = []
    doc = corpus_filtrato_embedding[0]
    for i in range(len(doc)):
        values.append(doc[i])
        

    for value in tqdm(values):
        for j in range(len(value)):
            if(j == len(value)-1):
                file.write(f"{value[j]}\n")
            else:
                file.write(f"{value[j]}, ")

## abbiamo quindi creato la feature matrix "X"

  0%|          | 0/499 [00:00<?, ?it/s]

Il codice appena scritto crea il .txt equivalente _A e il file equivalente a _node_attributes

Generalizziamo quanto fatto su un documento per inserire tutti i documenti!:

In [8]:
# Apre un file in modalità scrittura
with open('E3C_A.txt', 'w') as file:
    # Lista di coppie di valori
    values = []
    prev_count = 0
    for doc in corpus_filtrato_embedding:
        
        for i in range(len(doc)):
        
            if i == 0:
                val = (i+prev_count,i+1+prev_count)
                values.append(val)
            elif i == len(doc)-1:
                val = (i+prev_count,i-1+prev_count)
                values.append(val)
            else:
                val1= (i+prev_count,i-1+prev_count)
                values.append(val1)
                val2 = (i+prev_count,i+1+prev_count)
                values.append(val2)
        prev_count += len(doc)-1

    # Itera sulla lista e scrive le coppie nel file
    for value in values:
        file.write(f"{value[0]}, {value[1]}\n")

Così facendo abbiamo creato il file.A (lista di adiacenza)

In [9]:
# Apre un file in modalità scrittura
with open('E3C_graph_indicator.txt', 'w') as file:
    values = []
    for ind,doc in enumerate(corpus_filtrato_embedding):
        
        for i in range(len(doc)):
            values.append(ind)
        
    for value in values:
        file.write(f"{value}\n")

così facendo abbiamo creato il file .graph_indicator che dice quali nodi appartengono a quali grafi!

In [13]:
Doc0

array([[-0.11068563, -0.5326252 ,  0.86516297, ..., -0.06655114,
        -0.17851014,  0.21890694],
       [-0.51522696, -1.0387008 ,  1.1703496 , ..., -0.3740427 ,
        -0.5833328 , -0.22025317],
       [-0.6063753 ,  0.3164312 ,  1.7273585 , ...,  0.6947744 ,
        -0.9461107 , -0.25950187],
       ...,
       [-0.45306423, -0.6907194 ,  0.6036256 , ..., -0.3809749 ,
         0.02652384, -0.36844888],
       [-0.7124144 , -0.49716508,  1.0456138 , ..., -0.13115153,
        -0.5543687 ,  0.16673848],
       [-0.3750048 ,  0.30667436,  1.9497802 , ...,  0.2292165 ,
         0.03109064,  0.2720961 ]], dtype=float32)

In [23]:
with open('E3C_node_attributes.txt', 'w') as file:
    values = []
    for doc in tqdm(corpus_filtrato_embedding):
        for i in range(len(doc)):
            values.append(doc[i])
        

    for value in tqdm(values):
        for j in range(len(value)):
            if(j == len(value)-1):
                file.write(f"{value[j]}\n")
            else:
                file.write(f"{value[j]}, ")

  0%|          | 0/10213 [00:00<?, ?it/s]

  0%|          | 0/7699720 [00:00<?, ?it/s]

Così facendo abbiamo creato il file .node_attributes che ci dice le features di tutti i nodi (ovvero i coefficienti degli embedding delle parole)